In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as sstat

from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [ ]:
##MAXIMAL COUPLING (univariate)
def maximal_coupling(x,y):
      current_x = x
      current_y = y  
      proposed_x = current_x + tfd.Normal(0, 1).sample()
      pX = sstat.norm.pdf(proposed_x,current_x,1)
      w = tfd.Uniform(0,1).sample()*pX
      qX = sstat.norm.pdf(proposed_x,current_y,1)
      if w < qX:
          x_new = proposed_x
          y_new = proposed_x
      else:
          t = True
          while t == True:
                proposed_y = current_y + tfd.Normal(0, 1).sample()
                qY =  sstat.norm.pdf(proposed_y,current_y,1) #mi sa che vanno invertite le y anche se forse è uguale
                w =  tfd.Uniform(0,1).sample()*qY
                pY = sstat.norm.pdf(proposed_y,current_x,1)
                if w > pY :
                  t= False

          x_new = proposed_x
          y_new = proposed_y  
      return [x_new, y_new]

In [ ]:
def h(X):
  return X
  #funzione h di cui dobbiamo stimare la media

In [ ]:
#le x vanno da inizio a fine (compresi) e le y da inizio-1 a fine-1
def differenza(x,y,inizio,fine): 
    w=[] 
    for i in range(fine+1-inizio): 
        w.append(h(x[inizio+i])-h(y[inizio+i-1])) #4-3 + 6-2 + 5-1
    return w

#restituisce il vettore di h(x[t])-h(y[t-1])

In [ ]:
### TIME AVERAGED ESTIMATOR w_l=1/(m-k) l=k...m
m = 1000      #max(m,tau) number of iterations
k = 500        #burn-in

def time_averaged_est (m,k):

  x0 = tfd.Exponential(2).sample()
  y0 = tfd.Exponential(2).sample()

  x = []
  y = []
  H = []         #contiene gli m-k stimatori di cui faremo la media


  t=0
  tau = 1000000
  x.append(x0)     #sample from prior distribution
  y.append(y0)    #sample from prior distribution
  x.append(tfd.Normal(x0,0.5).sample())   #P transition matrix della MC NB: dalla congiunta
  t=t+1
  while t<max(m,tau):
     #[x[t+1],y[t]] = maximal_coupling(x[t],y[t-1])
     new_st = maximal_coupling(x[t],y[t-1])
     x.append(new_st[0])
     y.append(new_st[1])
     t=t+1
     if x[t]==y[t-1]:
         tau=t

  for l in range(m-k):
     inizio = k+l+1
     fine = tau-1
     vect = differenza(x,y,inizio,fine)
     H.append(h(x[k+l])+np.sum(vect))
     H_km = 1/(m-k+1)*np.sum(H)

  return H_km

In [ ]:
mmm = time_averaged_est(m,k)
mmm

21.8271222276364